In [1]:
import os
import sys
from Flight_Fare.exception import CustomException
from Flight_Fare.logger import logging
from Flight_Fare.config.configuration import Configuration
from Flight_Fare.entity.artifact_entity import *
from Flight_Fare.entity.config_entity import *
from Flight_Fare.constant import *
from Flight_Fare.util.util import *
from sklearn.impute import SimpleImputer

from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.pipeline import Pipeline
import pandas as pd 

f:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
f:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\venv\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
f:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\venv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [41]:
import pickle as pkl

In [275]:
train_df = pd.read_csv(r"Flight_Fare\artifact\data_ingestion\2023-07-25-01-27-30\ingested_data\train\Flight_Fare_Prediction.csv")
test_df = pd.read_csv(r"Flight_Fare\artifact\data_ingestion\2023-07-25-01-27-30\ingested_data\test\Flight_Fare_Prediction.csv")

In [276]:
X = train_df.drop(['Price','Additional_Info','Route'],axis = 1)

In [277]:
X

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops
0,Jet Airways,27/06/2019,Mumbai,Hyderabad,10:20,11:50,1h 30m,non-stop
1,Multiple carriers,3/03/2019,Delhi,Cochin,09:10,18:50,9h 40m,1 stop
2,Multiple carriers,27/05/2019,Delhi,Cochin,15:00,01:30 28 May,10h 30m,1 stop
3,Air Asia,9/05/2019,Kolkata,Banglore,22:40,10:40 10 May,12h,1 stop
4,Air India,9/06/2019,Delhi,Cochin,19:45,19:15 10 Jun,23h 30m,2 stops
...,...,...,...,...,...,...,...,...
7473,Jet Airways,27/03/2019,Delhi,Cochin,09:00,04:25 28 Mar,19h 25m,1 stop
7474,Jet Airways,9/05/2019,Kolkata,Banglore,14:05,20:45,6h 40m,1 stop
7475,Multiple carriers,15/05/2019,Delhi,Cochin,12:50,01:30 16 May,12h 40m,1 stop
7476,IndiGo,03/03/2019,Banglore,New Delhi,00:40,03:25,2h 45m,non-stop


In [278]:
num_cols = ['Dep_Time','Date_of_Journey','Arrival_Time','Duration']


class FeatureGenerator(BaseEstimator,TransformerMixin):

    def __init__(self,columns):
        self.columns = columns

    def fit(self,X,y=None):
        return self

    def transform(self,X,y=None):
        
            X['journey_day'] = X['Date_of_Journey'].str.split('/').str[0].astype(int)
            X['journey_month'] = X['Date_of_Journey'].str.split('/').str[1].str[1].astype(int)

        
            X['Dep_hr'] = X['Dep_Time'].str.split(':').str[0].astype(int)
            X['Dep_min'] = X['Dep_Time'].str.split(':').str[1].astype(int)

            X['Arrival_hr'] = X['Arrival_Time'].str.split(" ").str[0].str.split(':').str[0].astype(int)
            X['Arrival_min'] = X['Arrival_Time'].str.split(" ").str[0].str.split(':').str[1].astype(int)

            X['Duration_hr'] = 0
            X['Duration_min'] = 0
        
            

            duration = list(X['Duration'])

            for i in range(len(duration)):
                if 'h' not in duration[i]:
                    duration[i] = '0h ' + duration[i]
                    
                if 'm' not in duration[i]:
                    duration[i] = duration[i] + ' 0m'
                    
            for i in range(len(duration)):
                hr = int(duration[i].split("h")[0])
                min_ = int(duration[i].split(" ")[1].split("m")[0])
                X['Duration_hr'][i] = hr
                X['Duration_min'][i] = min_

            X['Duration_min'].fillna(0,inplace = True)

            X.drop(num_cols,inplace = True,axis =1 )
            
            return X

In [220]:
c = FeatureGenerator(num_cols)
c.fit(X)
c.transform(X)

,Airline,Source,Destination,Total_Stops,journey_day,journey_month,Dep_hr,Dep_min,Arrival_hr,Arrival_min,Duration_hr,Duration_min
0,Jet Airways,Mumbai,Hyderabad,non-stop,27,6,10,20,11,50,1,30
1,Multiple carriers,Delhi,Cochin,1 stop,3,3,9,10,18,50,9,40
2,Multiple carriers,Delhi,Cochin,1 stop,27,5,15,0,1,30,10,30
3,Air Asia,Kolkata,Banglore,1 stop,9,5,22,40,10,40,12,0
4,Air India,Delhi,Cochin,2 stops,9,6,19,45,19,15,23,30
...,...,...,...,...,...,...,...,...,...,...,...,...
7473,Jet Airways,Delhi,Cochin,1 stop,27,3,9,0,4,25,19,25
7474,Jet Airways,Kolkata,Banglore,1 stop,9,5,14,5,20,45,6,40
7475,Multiple carriers,Delhi,Cochin,1 stop,15,5,12,50,1,30,12,40
7476,IndiGo,Banglore,New Delhi,non-stop,3,3,0,40,3,25,2,45


In [279]:
cat_one = ['Airline','Source','Destination']
num_cols = ['Dep_Time','Date_of_Journey','Arrival_Time','Duration']
cat_label_coding = ['Total_Stops']

num_pipeline = Pipeline(steps=[
                ('feature_generator', FeatureGenerator(columns=num_cols)),
                ('scaler', StandardScaler())
            ]
            )

cat_pipeline = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder()),
        ('scaler', StandardScaler(with_mean=False))
]
)


preprocessing = ColumnTransformer(transformers = [
                ('num_pipeline', num_pipeline, num_cols),
                ('cat_pipeline', cat_pipeline, cat_one),
                ('cat_label',cat_pipeline,cat_label_coding)
            ])


In [274]:
preprocessing.fit(X)
preprocessing.transform(X)

array([[ 1.5826565 ,  1.10224748, -0.42808071, ...,  0.        ,
         2.13489406,  0.        ],
       [-1.25402435, -1.47195823, -0.60228552, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.5826565 ,  0.24417891,  0.44294333, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.16431607,  0.24417891, -0.07967109, ...,  0.        ,
         0.        ,  0.        ],
       [-1.25402435, -1.47195823, -2.17012879, ...,  0.        ,
         2.13489406,  0.        ],
       [-1.49041442,  1.10224748,  0.09453371, ...,  0.        ,
         0.        ,  0.        ]])

In [280]:
preprocessing.fit_transform(X)

array([[ 1.5826565 ,  1.10224748, -0.42808071, ...,  0.        ,
         2.13489406,  0.        ],
       [-1.25402435, -1.47195823, -0.60228552, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.5826565 ,  0.24417891,  0.44294333, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.16431607,  0.24417891, -0.07967109, ...,  0.        ,
         0.        ,  0.        ],
       [-1.25402435, -1.47195823, -2.17012879, ...,  0.        ,
         2.13489406,  0.        ],
       [-1.49041442,  1.10224748,  0.09453371, ...,  0.        ,
         0.        ,  0.        ]])

In [2]:
config = Configuration(CONFIG_FILE_PATH,CURRENT_TIME_STAMP)
data_transformation = config.get_data_transformation_config()
data_transformation.transformed_train_dir
base_name = "Flight_Fare_Prdiction.npz"
transformed_train_file_path = os.path.join(data_transformation.transformed_train_dir,base_name)

In [5]:
data_transformation.preprocessed_object_file_path

'f:\\Ineuron DS\\Internship\\FFP\\Flight_Fare_Predictions\\Flight_Fare\\artifact\\data_transformation\\2023-07-27-18-30-03\\preprocessed\\preprocessed.pkl'

In [35]:
model_trainer = config.get_model_trainer_config()
model_trainer.trained_model_file_path

'f:\\Ineuron DS\\Internship\\FFP\\Flight_Fare_Predictions\\Flight_Fare\\artifact\\model_trainer\\2023-07-27-01-19-40\\trained_model\\model.pkl'

In [15]:
d = {'dt':5,'rf':7,"xg":9,"ad":3,"lr":2}

scores = pd.DataFrame.from_dict(d,orient='index')

best_model = scores[scores[0] == max(scores[0])].index[0]

best_model_score = scores[scores[0] == max(scores[0])][0][0]

,0
dt,5
rf,7
xg,9
ad,3
lr,2


In [37]:
scores[scores[0] == max(scores[0])][0][0]

d['xg']

9

In [1]:
from Flight_Fare.util.util import evaluate_model



f:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
f:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\venv\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
f:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\venv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


0.6274765236618562

In [4]:
import numpy as np

c = float(np.float64(0.6274765236618562))

(c)

0.6274765236618562

In [22]:
from Flight_Fare.util.util import *

train_array = load_numpy_array_data(r"Flight_Fare\artifact\data_transformation\2023-07-27-17-27-50\transformed_data\train\Flight_Fare_Prediction.npz")
test_array = load_numpy_array_data(r"Flight_Fare\artifact\data_transformation\2023-07-27-17-27-50\transformed_data\test\Flight_Fare_Prediction.npz")

x_train,y_train,x_test,y_test = train_array[:,:-1],train_array[:,-1],test_array[:,:-1],test_array[:,-1]

In [24]:
from sklearn.linear_model import LinearRegression

models = {'Linear_reg': LinearRegression()}
            
param = {'Linear_reg': {}}

model_report,model_config,model_acc,model_rmse = evaluate_model(
X_train=x_train,X_test=x_test,
y_train=y_train,y_test=y_test,
models=models,param =param )

model_rmse

{'Linear_reg': [2844.3057018441227, 2765.333528242964]}

In [9]:
import pandas as pd

scores = pd.DataFrame.from_dict(model_report,orient='index')
best_model = scores[scores[0] == max(scores[0])].index[0]
best_model_score = scores[scores[0] == max(scores[0])][0][0]

best_model_score

0.6244248627604682

In [26]:
best_model_config = model_config[best_model]
model_accuracy = model_acc[best_model]
best_model_rmse = model_rmse[best_model]

In [28]:
best_model_rmse[0]

2844.3057018441227

In [72]:
from Flight_Fare.entity.artifact_entity import ModelTrainerArtifact
from Flight_Fare.component.data_transformation import DataTransformation

In [41]:
model_trainer_artifact[7]

Flight_Fare.entity.artifact_entity.ModelTrainerArtifact[7]

In [16]:
train_file_path = r"Flight_Fare\artifact\data_ingestion\2023-07-29-01-29-56\ingested_data\train\Flight_Fare_Prediction.csv"
test_file_path = r"Flight_Fare\artifact\data_ingestion\2023-07-29-01-29-56\ingested_data\test\Flight_Fare_Prediction.csv"

train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)

train_df.drop(['Additional_Info','Route'],inplace = True, axis = 1)
test_df.drop(['Additional_Info','Route'],inplace = True, axis = 1)

x = read_yaml_file(r"F:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\config\schema.yaml")
target_column = x['target_column']

y_train = np.array(train_df[target_column])
y_test = np.array(test_df[target_column])

train = train_df.drop(target_column,axis = 1)
test = test_df.drop(target_column,axis = 1)

X_train = np.array(train)
X_test = np.array(test)

In [15]:
[1,4,5,6]
[0,2,3,7]

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Price
0,Jet Airways,27/06/2019,Mumbai,Hyderabad,10:20,11:50,1h 30m,non-stop,8040
1,Multiple carriers,3/03/2019,Delhi,Cochin,09:10,18:50,9h 40m,1 stop,15135
2,Multiple carriers,27/05/2019,Delhi,Cochin,15:00,01:30 28 May,10h 30m,1 stop,13377
3,Air Asia,9/05/2019,Kolkata,Banglore,22:40,10:40 10 May,12h,1 stop,5192
4,Air India,9/06/2019,Delhi,Cochin,19:45,19:15 10 Jun,23h 30m,2 stops,9968
...,...,...,...,...,...,...,...,...,...
7473,Jet Airways,27/03/2019,Delhi,Cochin,09:00,04:25 28 Mar,19h 25m,1 stop,12242
7474,Jet Airways,9/05/2019,Kolkata,Banglore,14:05,20:45,6h 40m,1 stop,10844
7475,Multiple carriers,15/05/2019,Delhi,Cochin,12:50,01:30 16 May,12h 40m,1 stop,7670
7476,IndiGo,03/03/2019,Banglore,New Delhi,00:40,03:25,2h 45m,non-stop,6144


In [28]:
config = Configuration()
model_evaluation_file_path = config.get_model_evaluation_config().model_evaluation_file_path
pickle_file = r"Flight_Fare\artifact\model_trainer\2023-07-29-01-45-16\trained_model\model.pkl"
mode_acc = 0.6244248627604682


if os.path.exists(model_evaluation_file_path) == False:
    data = {'best_model':
            {'model_path': pickle_file,
             'model_accuracy': mode_acc,
             'time_stamp': CURRENT_TIME_STAMP
            }
        }
    os.makedirs(os.path.dirname(model_evaluation_file_path),exist_ok=True)
    with open(model_evaluation_file_path,"w") as file:
        yaml.dump(data,file)

with open(model_evaluation_file_path,"r") as file:
    v = yaml.safe_load(file)




In [29]:
print(v)

{'best_model': {'model_accuracy': 0.6244248627604682, 'model_path': 'Flight_Fare\\artifact\\model_trainer\\2023-07-29-01-45-16\\trained_model\\model.pkl', 'time_stamp': '2023-07-29-01-32-05'}}


In [91]:
config = Configuration()
model_evaluation_file_path = r"Flight_Fare\artifact\model_evaluation\model_evaluation.yaml"
pickle_file = r"Flight_Fare\artifact\model_trainer\2023-07-29-01-29-56\trained_model\model.pkl"
mode_acc = 0.6244248627604682

trained_modef_file_path = r"Flight_Fare\artifact\model_trainer\2023-07-29-01-45-16\trained_model\model.pkl"
model_accuracy = 0.7

if os.path.exists(model_evaluation_file_path) == False:
    data = {'best_model':
            {'model_path': pickle_file,
            'model_accuracy': model_accuracy,
            'time_stamp': CURRENT_TIME_STAMP
            }
        }
    os.makedirs(os.path.dirname(model_evaluation_file_path),exist_ok=True)
    with open(model_evaluation_file_path,"w") as file:
        yaml.dump(data,file)

with open(model_evaluation_file_path, "rb") as file_obj:
    model = yaml.safe_load(file_obj)
    if model['best_model']['model_accuracy'] < model_accuracy:
        data = {'best_model':
            {'model_path': trained_modef_file_path,
            'model_accuracy': model_accuracy,
            'time_stamp': CURRENT_TIME_STAMP
            }
        }
        yaml.dump(data,file)

In [ ]:
if model['best_model']['model_accuracy'] < model_accuracy:
        data = {'best_model':
            {'model_path': trained_modef_file_path,
            'model_accuracy': model_accuracy,
            'time_stamp': CURRENT_TIME_STAMP
            }
        }
        yaml.dump(data,file)

In [15]:
import shutil

config = Configuration()

evaluated_model_path = config.get_model_trainer_config().trained_model_file_path
export_dir = config.get_model_pusher_config().export_dir_path
model_file_name = os.path.basename(evaluated_model_path)

export_model_file_path = os.path.join(export_dir,model_file_name)

os.makedirs(export_model_file_path,exist_ok=True)

shutil.copy(evaluated_model_path,export_model_file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'f:\\Ineuron DS\\Internship\\FFP\\Flight_Fare_Predictions\\Flight_Fare\\artifact\\model_trainer\\2023-07-29-23-39-24\\trained_model\\model.pkl'

In [21]:
print(('None ' * 11))

None None None None None None None None None None None 


In [28]:
training_pipeline_file_path = config.get_training_pipeline_config().artifact_dir

EXPERIMENT_DIR_NAME,EXPERIMENT_FILE_NAME

experiment_file_path = os.path.join(training_pipeline_file_path,EXPERIMENT_DIR_NAME,EXPERIMENT_FILE_NAME)



'f:\\Ineuron DS\\Internship\\FFP\\Flight_Fare_Predictions\\Flight_Fare\\artifact\\experiment\\experiment.csv'

In [29]:
CURRENT_TIME_STAMP

'2023-07-29-23-39-24'

In [30]:
config.time_stamp

'2023-07-29-23-39-24'

In [3]:
from Flight_Fare.entity.artifact_entity import ModelTrainerArtifact

ModelTrainerArtifact.model_accuracy

_tuplegetter(7, 'Alias for field number 7')

In [7]:
import pandas as pd 

train = r"notebook\Data_Train.xlsx"

df = pd.read_excel(train)

In [9]:
df.columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price'],
      dtype='object')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [12]:
import opendatasets as os 

In [13]:
os.download("https://www.kaggle.com/datasets/nikhilmittal/flight-fare-prediction-mh")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading flight-fare-prediction-mh.zip to .\flight-fare-prediction-mh


100%|██████████| 626k/626k [00:01<00:00, 513kB/s]

In [23]:
mkdir -p ~/.kaggle


The syntax of the command is incorrect.


In [24]:
!copy kaggle.json ~/.kaggle/


The system cannot find the file specified.


In [25]:
!chmod 600 ~/.kaggle/kaggle.json

'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [26]:
!pip install cassandra-driver


  Obtaining dependency information for cassandra-driver from https://files.pythonhosted.org/packages/a2/44/2083985d2ade948bf18a9bfb1192e0462435c9e7db0ac9db4daf4d689c09/cassandra_driver-3.28.0-cp39-cp39-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB 1.3 MB/s eta 0:00:03
    --------------------------------------- 0.1/2.9 MB 812.7 kB/s eta 0:00:04
   - -------------------------------------- 0.1/2.9 MB 1.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.2/2.9 MB 1.1 MB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.9 MB 915.1 kB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.9 MB 1.1 MB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.9 MB 1.0 MB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.9 MB 1.0 MB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.9 MB 1.0 MB/s eta 0:00:03
   ---- ----------

In [29]:
import cassandra

In [30]:
print (cassandra.__version__)

3.28.0


In [31]:
!mkdir Flight_Fare_Dataset

In [32]:
cd Flight_Fare_Dataset

f:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\Flight_Fare_Dataset


In [40]:
!pip install git bash

ERROR: Could not find a version that satisfies the requirement git (from versions: none)
ERROR: No matching distribution found for git


In [4]:
!echo.>connect_database.py


In [5]:
!pip install Flask flask-cors cassandra-driver


  Obtaining dependency information for flask-cors from https://files.pythonhosted.org/packages/10/69/1e6cfb87117568a9de088c32d6258219e9d1ff7c131abf74249ef2031279/Flask_Cors-4.0.0-py2.py3-none-any.whl.metadata


In [6]:
FLASK_ENV=development
FLASK_APP=connect_database.py flask run


SyntaxError: invalid syntax (1388120224.py, line 1)

In [7]:
!pip3 install pulsar-client
import os
import time
import glob
from pulsar import Client, Producer
import csv

  Obtaining dependency information for pulsar-client from https://files.pythonhosted.org/packages/16/41/916913948556521ee29c44a97d40b60df66dc0e6de09103552011663d10c/pulsar_client-3.2.0-cp39-cp39-win_amd64.whl.metadata
   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.4 MB 1.4 MB/s eta 0:00:03
   - -------------------------------------- 0.1/3.4 MB 939.4 kB/s eta 0:00:04
   -- ------------------------------------- 0.2/3.4 MB 1.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.2/3.4 MB 1.2 MB/s eta 0:00:03
   -- ------------------------------------- 0.2/3.4 MB 1.2 MB/s eta 0:00:03
   ----- ---------------------------------- 0.5/3.4 MB 1.5 MB/s eta 0:00:02
   ------ --------------------------------- 0.5/3.4 MB 1.3 MB/s eta 0:00:03
   -------- ------------------------------- 0.7/3.4 MB 1.5 MB/s eta 0:00:02
   --------- ------------------------------ 0.8/3.4 MB 1.6 MB/s eta 0:00:02
   --------- --------------

In [11]:
POLLING_INTERVAL = 5  # Polling interval in seconds
LOCAL_DIRECTORY = 'F:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\Flight_Fare_Dataset'
CSV_FILE_PATTERN = '*.csv'
ASTRA_STREAMING_SERVICE_URL = 'pulsar+ssl://cluster.region.signalpath.astra.datastax.com:6651'
ASTRA_STREAMING_TOKEN = '"AstraCS:sdYaYcSuNdZGyehCcvzkaJWw:42e616abccf5e72d545684387b5e494406d144c7375f536cfc96210d3b3bd9c3"'
PULSAR_TOPIC = 'bin/pulsar-admin topics create-partitioned-topic apache/pulsar/test-topic -p 4'


# Authentication using Astra Streaming token
from pulsar import AuthenticationToken
auth = AuthenticationToken(ASTRA_STREAMING_TOKEN)

client = Client(ASTRA_STREAMING_SERVICE_URL, authentication=auth)
producer = client.create_producer(PULSAR_TOPIC)

def send_csv_to_pulsar(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            csv_line = ','.join(row)
            producer.send(csv_line.encode('utf-8'))
            print(f'Sent line: {csv_line} to Pulsar topic: {PULSAR_TOPIC}')


def main():
    while True:
        csv_files = glob.glob(os.path.join(LOCAL_DIRECTORY, CSV_FILE_PATTERN))
        for file_path in csv_files:
            send_csv_to_pulsar(file_path)
            os.remove(file_path)  # Remove the file after sending to avoid processing it again
        time.sleep(POLLING_INTERVAL)

InvalidTopicName: Pulsar error: InvalidTopicName

In [10]:
!bin/pulsar-admin topics create-partitioned-topic apache/pulsar/test-topic -p 4

'bin' is not recognized as an internal or external command,
operable program or batch file.


In [58]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config= {'secure_connect_bundle': r'C:\Users\HP-LAPTOP\Downloads\secure-connect-flight-fare.zip'}
auth_provider = PlainTextAuthProvider('FidRdxHUPYnyAPYZvbFTtuKm', "NfIoNH8zyuKRLWI4Cn82pJNZEy7KlGt5,qPobB2SC._yHl6,pCRlPmPgZlLtKXmCrB47IM3QcuZBj5XNxPZgKDrvtoEsgzlyAY1vUc0.ZPchc_C0elKsnmH739wWaxyi")
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

4.0.0.6816


In [64]:
row = session.execute("use predict").one()

In [65]:
row = session.execute("SELECT * FROM train;")

In [66]:
x = row.all()

In [72]:
import pandas as pd

df = pd.DataFrame([d for d in x])

In [73]:
df

,Route,Arrival_Time,Dep_Time,Destination,Source,Airline,Date_of_Journey,Duration,Total_Stops,Price,Additional_Info
0,CCU → BOM → HBX → BLR,17:20 16 May,09:25,Banglore,Kolkata,Air India,15/05/2019,31h 55m,2 stops,13616,No info
1,CCU → BOM → HBX → BLR,17:20 19 May,09:25,Banglore,Kolkata,Air India,18/05/2019,31h 55m,2 stops,13616,No info
2,CCU → BOM → HBX → BLR,18:20,09:25,Banglore,Kolkata,Air India,1/05/2019,8h 55m,2 stops,8366,No info
3,CCU → BOM → HBX → BLR,18:20,09:25,Banglore,Kolkata,Air India,15/05/2019,8h 55m,2 stops,8366,No info
4,CCU → BOM → HBX → BLR,18:20,09:25,Banglore,Kolkata,Air India,18/05/2019,8h 55m,2 stops,8366,No info
...,...,...,...,...,...,...,...,...,...,...,...
10457,DEL → MAA → COK,22:05,15:45,Cochin,Delhi,SpiceJet,24/06/2019,6h 20m,1 stop,5583,No info
10458,DEL → MAA → COK,22:05,15:45,Cochin,Delhi,SpiceJet,6/06/2019,6h 20m,1 stop,5583,No info
10459,DEL → MAA → COK,22:05,15:45,Cochin,Delhi,SpiceJet,9/06/2019,6h 20m,1 stop,6297,No info
10460,BLR → STV → DEL,15:25,11:20,New Delhi,Banglore,IndiGo,09/03/2019,4h 5m,1 stop,5746,No info


In [3]:
import os

In [5]:
basename = 'Accuracy.yaml'
acc_file_path = os.path.join(ROOT_DIR,'Accuracy',basename)
metric = {'model_accuracy':'model_accuracy'}

os.makedirs(os.path.dirname(acc_file_path),exist_ok=True)
write_yaml_file(acc_file_path,metric)
index_number = 0

In [6]:
acc_file_path = os.path.join(ROOT_DIR,'Accuracy','Accuracy.yaml')
acc = read_yaml_file(acc_file_path)
accuracy = acc['model_accuracy']

In [7]:
accuracy

0.6203448858737616

In [1]:
import uuid


experiment_id = str(uuid.uuid4())

In [5]:
if  experiment_id is not None:
    print("y")

y


In [1]:
from Flight_Fare.entity.Flight_Fare_Predictor import Flight_Data,FarePredictor

import sys

import pip
from Flight_Fare.util.util import read_yaml_file, write_yaml_file
from Flight_Fare.logger import logging
from Flight_Fare.exception import CustomException
import os, sys
import json
from Flight_Fare.config.configuration import Configuration
from Flight_Fare.constant import CONFIG_DIR, CURRENT_TIME_STAMP
from Flight_Fare.pipeline.pipeline import Pipeline


ROOT_DIR = os.getcwd()
LOG_FOLDER_NAME = "logs"
PIPELINE_FOLDER_NAME = "Flight_Fare"
SAVED_MODELS_DIR_NAME = "saved_models"
LOG_DIR = os.path.join(ROOT_DIR, LOG_FOLDER_NAME)
PIPELINE_DIR = os.path.join(ROOT_DIR, PIPELINE_FOLDER_NAME)
MODEL_DIR = os.path.join(ROOT_DIR, SAVED_MODELS_DIR_NAME)

f:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
f:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\venv\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
f:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\venv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
f:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [3]:
lat = os.listdir(MODEL_DIR)[-1]
latest = os.path.join(MODEL_DIR,lat)

In [30]:
os.listdir(latest)[0]

'model.pkl'

In [28]:
latest

'f:\\Ineuron DS\\Internship\\FFP\\Flight_Fare_Predictions\\saved_models\\2023-08-02-19-04-335'

In [45]:
folder_name = os.listdir(MODEL_DIR)[-1]
latest_folder_dir = os.path.join(MODEL_DIR,folder_name)
folder_name = os.listdir(latest_folder_dir)[0]
latest_folder_dir = os.path.join(latest_folder_dir, folder_name)
file_name = os.listdir(latest_folder_dir)[0]
latest_model_path = os.path.join(latest_folder_dir,file_name)

In [47]:
with open (latest_model_path,'r') as file:
    print(x)

NameError: name 'x' is not defined

In [8]:
F:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\saved_models\2023-08-02-22-58-47\model\model.pkl

NameError: name 'x' is not defined

In [ ]:
F:\Ineuron DS\Internship\FFP\Flight_Fare_Predictions\saved_models\2023-08-02-19-04-33\model.pkl\model.pkl

In [9]:
latest_model_path

'f:\\Ineuron DS\\Internship\\FFP\\Flight_Fare_Predictions\\saved_models\\2023-08-02-19-04-33\\model.pkl'

In [11]:
import os
import sys
from Flight_Fare.exception import CustomException
from Flight_Fare.logger import logging

from Flight_Fare.config.configuration import Configuration
from Flight_Fare.entity.artifact_entity import *
from Flight_Fare.entity.config_entity import *
from Flight_Fare.constant import *
from Flight_Fare.util.util import *

In [25]:
config = Configuration(CONFIG_FILE_PATH,CURRENT_TIME_STAMP)

evaluated_model_path = config.get_model_trainer_config().trained_model_file_path
export_dir = config.get_model_pusher_config().export_dir_path
model_file_name = os.path.basename(evaluated_model_path)
model_file_name = model_file_name.split(".")[0]
export_model_file_path = os.path.join(export_dir,model_file_name)

In [24]:
model_file_name.split(".")[0]

'model'

In [26]:
export_model_file_path

'f:\\Ineuron DS\\Internship\\FFP\\Flight_Fare_Predictions\\saved_models\\2023-08-02-22-37-32\\model'

In [50]:
import pandas as pd
df = pd.read_excel(r"F:\Ineuron DS\Internship\Flight Fare Predicitons\notebook\Data_Train.xlsx")

In [ ]:
df